<a href="https://colab.research.google.com/github/lupinheiro1/imersao-alura-google-public/blob/main/Projeto_Alura_e_Google_Achar_Prontos_socorros_perto_da_sua_localiza%C3%A7%C3%A3o_Luiza_Pinheiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install google-genai google-adk googlemaps

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━

In [17]:
import os
from google.colab import userdata
os.environ ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

MAPS_API_KEY = userdata.get('MinhaAPIKeyImersao')

In [3]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [21]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from IPython.display import display, Markdown
import textwrap
import googlemaps
gmaps = googlemaps.Client(key=MAPS_API_KEY)

In [7]:
# --- Funções Auxiliares ---
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
            final_response += "\n"
    return final_response

def to_markdown(text): # Função auxiliar para exibir Markdown
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '', predicate=lambda line: True))

In [8]:
# --- Agente 1: Coletor de Dados de Saúde ---
def agente_coletor_saude(localizacao_usuario, raio_busca=20000):
    print(f"--- Agente Coletor de Saúde: Buscando pronto-socorros próximos a '{localizacao_usuario}' ---")

    try:
        places_result = gmaps.places(query='pronto-socorro', location=localizacao_usuario, radius=raio_busca, language='pt-BR')

        pronto_socorros = []
        for place in places_result['results']:
            nome = place.get('name', 'Nome não disponível')
            endereco = place.get('formatted_address', 'Endereço não disponível')
            avaliacao = place.get('rating', 'Sem avaliação')
            place_id = place.get('place_id')

            telefone = 'Não disponível'
            if place_id:
                details = gmaps.place(place_id=place_id, fields=['international_phone_number'])
                telefone = details['result'].get('international_phone_number', 'Não disponível')

            lotacao = "Não disponível via API pública"
            # Para o MVP, mas a ideia é conseguirmos icluir esse dado, pois é fundamental na decisão das mães

            pronto_socorros.append({
                'nome': nome,
                'endereco': endereco,
                'telefone': telefone,
                'avaliacao': avaliacao,
                'lotacao': lotacao,
                'place_id': place_id
            })

        return pronto_socorros

    except Exception as e:
        print(f"Erro no Agente Coletor de Saúde: {e}")
        return []

In [9]:
# --- Agente 2: Buscador de Convênios (Agente google-adk) ---
def agente_buscador_convenios(nome_hospital):
    print(f"\n--- Agente Buscador de Convênios: Buscando convênios para '{nome_hospital}' ---")

    buscador_convenios = Agent(
        name="agente_buscador_convenios",
        model=MODEL_ID,
        instruction=f"""Você é um assistente especializado em encontrar informações sobre convênios médicos.
        Sua tarefa é usar a ferramenta de busca do google (google_search) para pesquisar quais convênios o hospital '{nome_hospital}' aceita.
        Retorne as informações mais relevantes que você encontrar, incluindo links para páginas oficiais ou de convênios do hospital.
        Seja conciso, mas informativo.
        """,
        description="Agente que busca informações de convênios aceitos por hospitais no Google.",
        tools=[google_search]
    )

    entrada_do_agente = f"Quais convênios o hospital {nome_hospital} aceita?"
    resultado_busca_convenios = call_agent(buscador_convenios, entrada_do_agente)

    return resultado_busca_convenios

In [10]:
# --- Agente 3: Apresentador de Informações (Função Python Tradicional) ---
def agente_apresentador_informacoes(hospitais_com_dados_completos):
    print("\n--- Agente Apresentador de Informações: Consolidando Resultados ---")

    if not hospitais_com_dados_completos:
        print("Nenhum pronto-socorro encontrado ou processado.")
        return

    markdown_output = ""
    for i, hospital in enumerate(hospitais_com_dados_completos):
        markdown_output += f"### {i+1}. {hospital['nome']}\n"
        markdown_output += f"- **Endereço:** {hospital['endereco']}\n"
        markdown_output += f"- **Telefone:** {hospital['telefone']}\n"
        markdown_output += f"- **Avaliação (Google):** {hospital['avaliacao']}/5.0\n"
        markdown_output += f"- **Lotação:** {hospital['lotacao']}\n" # Placeholder por enquanto
        markdown_output += f"- **Convênios (Via Google Search):**\n"

        # Aqui, o resultado_convenios já vem formatado pelo Agente 2
        # É importante que o Agente 2 seja instruído a retornar algo legível
        markdown_output += textwrap.indent(hospital['convenios_info'], '  ') + "\n\n"

    display(to_markdown(markdown_output))

In [24]:
# --- Lógica Principal de Execução ---
if __name__ == "__main__":
    localizacao_usuario = input("Por favor, digite sua localização com nome da cidade (ex: 'Praça Universitária, Setor Leste Universitário, Goiânia'): ")

    # 1. Executa o Agente Coletor de Saúde (Agente 1)
    pronto_socorros_iniciais = agente_coletor_saude(localizacao_teste)

    hospitais_com_info_completa = []
    if pronto_socorros_iniciais:
        for hospital in pronto_socorros_iniciais:
            # 2. Para cada hospital, executa o Agente Buscador de Convênios (Agente 2)
            nome_hospital = hospital['nome']
            convenios_info = agente_buscador_convenios(nome_hospital)

            # Adiciona a informação de convênios ao dicionário do hospital
            hospital['convenios_info'] = convenios_info
            hospitais_com_info_completa.append(hospital)

    # 3. Executa o Agente Apresentador de Informações (Agente 3)
    agente_apresentador_informacoes(hospitais_com_info_completa)

Por favor, digite sua localização com nome da cidade (ex: 'Praça Universitária, Setor Leste Universitário, Goiânia'): Praça Universitária, Setor Leste Universitário, Goiânia, GO
--- Agente Coletor de Saúde: Buscando pronto-socorros próximos a 'Praça universitaria setor leste universitario' ---

--- Agente Buscador de Convênios: Buscando convênios para 'Complexo Hospitalar Padre Bento' ---

--- Agente Buscador de Convênios: Buscando convênios para 'Pronto-Socorro do Hospital BP' ---

--- Agente Apresentador de Informações: Consolidando Resultados ---


### 1. Complexo Hospitalar Padre Bento
- **Endereço:** Av. Emílio Ribas, 1819 - Gopouva, Guarulhos - SP, 07051-000, Brasil
- **Telefone:** +55 11 2463-5650
- **Avaliação (Google):** 3.5/5.0
- **Lotação:** Não disponível via API pública
- **Convênios (Via Google Search):**
  O Complexo Hospitalar Padre Bento em Guarulhos atende pelo Sistema Único de Saúde (SUS). Além disso, há informações sobre convênios e contratos específicos firmados pelo hospital com diversas entidades para gerenciamento de serviços e leitos, como o CEJAM.

  Para informações mais precisas sobre quais planos de saúde são aceitos atualmente, é recomendável entrar em contato diretamente com o hospital.



### 2. Pronto-Socorro do Hospital BP
- **Endereço:** R. Maestro Cardim, 637 - Bela Vista, São Paulo - SP, 01323-001, Brasil
- **Telefone:** +55 11 3505-1000
- **Avaliação (Google):** 2.9/5.0
- **Lotação:** Não disponível via API pública
- **Convênios (Via Google Search):**
  O Hospital BP (Beneficência Portuguesa) aceita diversos convênios médicos em seu pronto-socorro e demais unidades. Para verificar a cobertura do seu plano, você pode:

  1.  **Consultar a lista de convênios atendidos no site da BP:** No site, você pode selecionar a unidade de atendimento (BP Mirante ou Hospital BP - Unidade Paulista) e verificar os convênios aceitos em cada uma delas.
  2.  **Entrar em contato com a Central de Atendimento da BP:** O telefone para contato é (11) 3505-1000.
  3.  **Verificar diretamente com a sua operadora de saúde:** Confirme se o seu plano oferece cobertura para os serviços que você precisa no Hospital BP.

  Alguns dos convênios aceitos pelo Hospital BP incluem:

  *   SulAmérica Saúde
  *   Amil
  *   Bradesco Saúde
  *   Unimed
  *   Allianz Saúde

  Para obter uma lista mais completa e informações atualizadas, recomendo consultar os links fornecidos e entrar em contato diretamente com o hospital ou sua operadora de saúde.



